# Etape 2: Exemple du Ping6

Nous ouvrons le fichier *ping6-total.pcapng* avec Wireshark. Un filtre `icmpv6` est appliqué pour afficher uniquement les paquets de Neighbor Discovery ainsi que les échanges Echo (request + reply). Le résultat est enregistré dans un fichier *ping6-display.pcapng*  


Machine source (initie le ping) : 2001:660:6701:30cc:84fc:c335:133c:f204  
Machine cible : 2001:660:6701:30cc::1

Trame ND Neighbor Solicitation :  
•En-tête Ethernet  
• Adresse MAC Source: 64:00:6a:6a:c4:01  
• Adresse MAC Destination: 33:33:ff:00:00:01  
•En-tête IPv6  
• Adresse IP Source: 2001:660:6701:30cc:84fc:c335:133c:f204  
• Adresse IP Destination: ff02::1:ff00:1  
•En-tête du paquet ICMP/ND  
• Adresse IP Cible: 2001:660:6701:30cc::1  
• Adresse MAC: 64:00:6a:6a:c4:01  

In [2]:
from scapy.all import *
trames=rdpcap("capture/ping6-display.pcapng")
print("Voici les trames capturées :\n")
trames.show()

Voici les trames capturées :

0000 Ether / IPv6 / ICMPv6ND_NS / ICMPv6 Neighbor Discovery Option - Source Link-Layer Address 64:00:6a:6a:c4:01
0001 Ether / IPv6 / ICMPv6ND_NA / ICMPv6 Neighbor Discovery Option - Destination Link-Layer Address 00:13:1a:a3:e1:18
0002 Ether / IPv6 / ICMPv6 Echo Request (id: 0x4 seq: 0x1)
0003 Ether / IPv6 / ICMPv6 Echo Reply (id: 0x4 seq: 0x1)
0004 Ether / IPv6 / ICMPv6 Echo Request (id: 0x4 seq: 0x2)
0005 Ether / IPv6 / ICMPv6 Echo Reply (id: 0x4 seq: 0x2)
0006 Ether / IPv6 / ICMPv6 Echo Request (id: 0x4 seq: 0x3)
0007 Ether / IPv6 / ICMPv6 Echo Reply (id: 0x4 seq: 0x3)
0008 Ether / IPv6 / ICMPv6 Echo Request (id: 0x4 seq: 0x4)
0009 Ether / IPv6 / ICMPv6 Echo Reply (id: 0x4 seq: 0x4)
0010 Ether / IPv6 / ICMPv6 Echo Request (id: 0x4 seq: 0x5)
0011 Ether / IPv6 / ICMPv6 Echo Reply (id: 0x4 seq: 0x5)
0012 Ether / IPv6 / ICMPv6 Echo Request (id: 0x4 seq: 0x6)
0013 Ether / IPv6 / ICMPv6 Echo Reply (id: 0x4 seq: 0x6)


In [ ]:
# On affiche les infos de la 1er trame (les infos de toutes les couches en partant de la couche 2)
trames[0]

<Ether  dst=33:33:ff:00:00:01 src=64:00:6a:6a:c4:01 type=IPv6 |<IPv6  version=6 tc=0 fl=0 plen=32 nh=ICMPv6 hlim=255 src=2001:660:6701:30cc:84fc:c335:133c:f204 dst=ff02::1:ff00:1 |<ICMPv6ND_NS  type=Neighbor Solicitation code=0 cksum=0x1d60 res=0 tgt=2001:660:6701:30cc::1 |<ICMPv6NDOptSrcLLAddr  type=1 len=1 lladdr=64:00:6a:6a:c4:01 |>>>>

In [ ]:
# On affiche les infos du 1er paquet (les infos de toutes les couches en partant de la couche 3)
trames[0][IPv6]

<IPv6  version=6 tc=0 fl=0 plen=32 nh=ICMPv6 hlim=255 src=2001:660:6701:30cc:84fc:c335:133c:f204 dst=ff02::1:ff00:1 |<ICMPv6ND_NS  type=Neighbor Solicitation code=0 cksum=0x1d60 res=0 tgt=2001:660:6701:30cc::1 |<ICMPv6NDOptSrcLLAddr  type=1 len=1 lladdr=64:00:6a:6a:c4:01 |>>>

In [ ]:
# On affiche l'IP de destination du 1er paquet
trames[0][IPv6].dst # On obtient l'IPv6 multicast local de destination du paquet ND NS 

'ff02::1:ff00:1'

## Automatisation du processus ICMPv6

L’ensemble des trames capturées dans le fichier pcap est chargé dans la variable `trames`.  
Pour chaque trame, on sélectionne (via des conditions if) uniquement les paquets IPv6 contenant un message ICMPv6 (Next Header = 58) ET dont le type est Neighbor Solicitation (type = 135).  
Les informations utiles sont ensuite extraites.  
Dans notre exemple, un seul paquet est retenu.

In [3]:
listetrames = rdpcap("capture/ping6-total.pcapng")
for trame in listetrames:
    try:
        if (trame.version) == 6: # C'est de l'IPv6 ?
            if (trame[IPv6].nh) == 58: # C'est de l'ICMPv6 ?
                if (trame[0][2].type) == 135: # C'est une ND NS ?
                    print(f"Ethernet: MAC Source : {trame[0][0].src}")
                    print(f"Ethernet: MAC Destination : {trame[0][0].dst}")
                    print(f"IPv6 : IP Source : {trame[0][1].src}")
                    print(f"IPv6 : IP Destination : {trame[0][1].dst}")
                    print(f"ICMPv6 : IP Cible : {trame[0][2].tgt}")
                    print(f"ICMPv6 : MAC Demandeur : {trame[0][3].lladdr}")
    except Exception: # Si c'est un paquet ARP ou autre qui n'a pas de champ version, on ignore et passe au prochain paquet
        pass

Ethernet: MAC Source : 64:00:6a:6a:c4:01
Ethernet: MAC Destination : 33:33:ff:00:00:01
IPv6 : IP Source : 2001:660:6701:30cc:84fc:c335:133c:f204
IPv6 : IP Destination : ff02::1:ff00:1
ICMPv6 : IP Cible : 2001:660:6701:30cc::1
ICMPv6 : MAC Demandeur : 64:00:6a:6a:c4:01


# Sniffer le reseau en temps reél

In [ ]:
from scapy.all import *
ICMPv6_types = {128: 'Echo-Request', 129: 'Echo-Reply', 135: 'Neighbor Solicitation', 136: 'Neighbor Advertisement', 133: 'Router Solicitation', 134: 'Router Advertisement'}
def print_icmpv6(trame): # Fonction pour afficher les informations de la trame
    print(trame.summary()) # On affiche la liste des encapsulations du paquet (Ethernet/IPv6/ICMPv6ND_NS)
    type = trame[2].type # Dans la couche IP, on extrait le type de données (au-dessus de la couche IP)
    if (type == 135 or type == 136): # Si le type est 135 ou 136, c'est-à-dire si c'est un "Neighbor Solicitation" ou "Neighbor Advertisement"
        print(f"TYPE PACKET ICMP : {ICMPv6_types[type]}") # Type de message ICMP
        print(f"Ethernet: MAC Source : {trame[0].src}") # Adresse MAC source en accédant à la couche Ethernet.
        print(f"Ethernet: MAC Destination : {trame[0].dst}") # Adresse MAC destination en accédant à la couche Ethernet.
        print(f"IPv6 : IP Source : {trame[1].src}") # Adresse source IPv6 en accédant à la couche IPv6.
        print(f"IPv6 : IP Destination : {trame[1].dst}") # Adresse de destination IPv6 en accédant à la couche IPv6.
        print(f"ICMPv6 : IP Cible : {trame[2].tgt}") # Adresse cible (target) demandée dans le paquet ICMPv6.
        print(f"ICMPv6 : MAC Demandeur : {trame[2].lladdr}") # Adresse MAC demandée (par Neighbor Solicitation) fournie par le Neighbor Advertisement
        print("\n") # Retour à la ligne
    else: # Sinon, si c'est un message ICMP "autre"
        print(f"TYPE PACKET ICMP : {ICMPv6_types[type]}") # Type de message ICMP
        print(f"Ethernet: MAC Source : {trame[0].src}") # Adresse MAC source en accédant à la couche Ethernet.
        print(f"Ethernet: MAC Destination : {trame[0].dst}") # Adresse MAC destination en accédant à la couche Ethernet.
        print(f"IPv6 : IP Source : {trame[1].src}") # Adresse de destination IPv6 en accédant à la couche IPv6.
        print(f"IPv6 : IP Destination : {trame[1].dst}") # Adresse destination IPv6 du paquet.
        print("\n")
carte = conf.iface # On récupère l'interface de l'ordinateur
print(f"On commence le 'sniffing' sur la carte {carte}:")
print("\n")
# On sniffe les paquets avec un filtre (uniquement IPV6/ICMP), on récupère 6 paquets.
# Pour chaque paquet, on appelle la fonction "print_icmp"
sniff(filter="ip6 proto 58", prn=print_icmpv6, store=0, iface=carte, count=6)

On commence le 'sniffing' sur la carte \Device\NPF_{F5E07108-4D29-43ED-AF37-D6696A4CFC11}:


Ether / IPv6 / ICMPv6 Echo Request (id: 0x1 seq: 0x1)
TYPE PACKET ICMP : Echo-Request
Ethernet: MAC Source : 3c:0a:f3:24:fd:c7
Ethernet: MAC Destination : 10:e9:92:9a:35:f0
IPv6 : IP Source : fe80::208d:7fe9:28da:4e07
IPv6 : IP Destination : fe80::12e9:92ff:fe9a:35f0


Ether / IPv6 / ICMPv6ND_NS / ICMPv6 Neighbor Discovery Option - Source Link-Layer Address 10:e9:92:9a:35:f0
TYPE PACKET ICMP : Neighbor Solicitation
Ethernet: MAC Source : 10:e9:92:9a:35:f0
Ethernet: MAC Destination : 33:33:ff:da:4e:07
IPv6 : IP Source : fe80::12e9:92ff:fe9a:35f0
IPv6 : IP Destination : ff02::1:ffda:4e07
ICMPv6 : IP Target : fe80::208d:7fe9:28da:4e07
ICMPv6 : MAC Requested : 10:e9:92:9a:35:f0


Ether / IPv6 / ICMPv6ND_NA / ICMPv6 Neighbor Discovery Option - Destination Link-Layer Address 3c:0a:f3:24:fd:c7
TYPE PACKET ICMP : Neighbor Advertisement
Ethernet: MAC Source : 3c:0a:f3:24:fd:c7
Ethernet: MAC Destination 

<Sniffed: TCP:0 UDP:0 ICMP:0 Other:0>

On observe ici un échange de type ping entre deux machines :  
- Le 2ᵉ paquet est une Neighbor Solicitation : `fe80::12e9:92ff:fe9a:35f0` demande l’adresse MAC de `fe80::208d:7fe9:28da:4e07` en diffusant un message ciblé.  
- Le 3ᵉ paquet est une Neighbor Advertisement : `fe80::208d:7fe9:28da:4e07` répond à `fe80::12e9:92ff:fe9a:35f0` en communiquant son adresse MAC `3c:0a:f3:24:fd:c7`.  
- Les paquets suivants sont les messages ICMP Echo Request et Echo Reply (le ping).

## Automatisation du processus IPv4
Même principe que précédemment, mais cette fois avec des trames IPv4, à partir du fichier *ping4-total.pcapng*.

In [ ]:
from scapy.all import *
listetrames = rdpcap("capture/ping4-total.pcapng")
for trame in listetrames:
    if trame[0].type == 0x0806: # Message ARP
        if trame[1].op == 1: # Si le champ opération est à 1 alors c'est une ARP request
            print(f"{trame[1].psrc} ({trame[1].hwsrc}) demande l'adresse MAC de {trame[1].pdst} (requête ARP)")
        else: # Sinon le champ est à 2, c'est une ARP reply.
            print(f"{trame[1].psrc} répond en lui fournissant son adresse MAC {trame[1].hwsrc} à {trame[1].pdst} ({trame[1].hwdst}) (réponse ARP)")
    elif trame[0].type == 0x0800: # Message IPv4
        if trame[1].proto == 1: # Message ICMP
            if trame[2].type == 8: # ICMP ping request
                print(f"{trame[1].src} envoie un ping à {trame[1].dst}")
            elif trame[2].type == 0: # ICMP ping reply
                print(f"{trame[1].src} répond au ping de {trame[1].dst}")

192.168.2.16 (3c:0a:f3:24:fd:c7) demande l'adresse mac de 192.168.2.12 (requette ARP)
192.168.2.12 repond en lui fournissant sont adresse mac 9c:9c:1f:e8:e9:1c a 192.168.2.16 (3c:0a:f3:24:fd:c7) (reponse ARP)
192.168.2.16 envoie un ping a 192.168.2.12
192.168.2.12 repond aux ping a 192.168.2.16
192.168.2.16 envoie un ping a 192.168.2.12
192.168.2.12 repond aux ping a 192.168.2.16
192.168.2.16 envoie un ping a 192.168.2.12
192.168.2.12 repond aux ping a 192.168.2.16
192.168.2.16 envoie un ping a 192.168.2.12
192.168.2.12 repond aux ping a 192.168.2.16


# Sniffer le reseau en temps réel 
On reprend le programme précédent et on l’adapte dans une fonction. Cette fonction prend des trames Ethernet en entrée et retourne un message formaté proprement, uniquement si le paquet est de type ICMPv4 ou ARP.

In [ ]:
from scapy.all import *
def print_icmpv4(trame): # Fonction pour afficher les informations de la trame
    if trame[0].type == 0x0806: # Message ARP
        if trame[1].op == 1: # Si le champ opération est à 1 alors c'est une ARP request
            print(f"{trame[1].psrc} ({trame[1].hwsrc}) demande l'adresse MAC de {trame[1].pdst} (requête ARP)")
        else: # Sinon le champ est à 2, c'est une ARP reply.
            print(f"{trame[1].psrc} répond en lui fournissant son adresse MAC {trame[1].hwsrc} à {trame[1].pdst} ({trame[1].hwdst}) (réponse ARP)")
    elif trame[0].type == 0x0800: # Message IPv4
        if trame[1].proto == 1: # Message ICMP
            if trame[2].type == 8: # ICMP ping request
                print(f"{trame[1].src} envoie un ping à {trame[1].dst}")
            elif trame[2].type == 0: # ICMP ping reply
                print(f"{trame[1].src} répond au ping de {trame[1].dst}")
carte = conf.iface # On récupère l'interface de l'ordinateur
print(f"On commence le 'sniffing' sur la carte {carte}:")
print("\n")
# On sniffe toutes les trames
# Pour chaque trame, on appelle la fonction "print_icmp"
sniff(prn=print_icmpv4, store=0, iface=carte)

On commence le 'sniffing' sur la carte \Device\NPF_{F5E07108-4D29-43ED-AF37-D6696A4CFC11}:


192.168.2.16 (3c:0a:f3:24:fd:c7) demande l'adresse mac de 192.168.2.12 (requette ARP)
192.168.2.16 (3c:0a:f3:24:fd:c7) demande l'adresse mac de 192.168.2.12 (requette ARP)
192.168.2.12 repond en lui fournissant sont adresse mac 9c:9c:1f:e8:e9:1c a 192.168.2.16 (3c:0a:f3:24:fd:c7) (reponse ARP)
192.168.2.16 envoie un ping a 192.168.2.12
192.168.2.12 repond aux ping a 192.168.2.16
192.168.2.16 envoie un ping a 192.168.2.12
192.168.2.12 repond aux ping a 192.168.2.16
192.168.2.16 envoie un ping a 192.168.2.12
192.168.2.12 repond aux ping a 192.168.2.16
192.168.2.16 envoie un ping a 192.168.2.12
192.168.2.12 repond aux ping a 192.168.2.16


<Sniffed: TCP:0 UDP:0 ICMP:0 Other:0>

Dans les deux cas (capture en direct ou fichier), on observe :
- Une requête ARP émise par 192.168.2.16, qui cherche à découvrir l’adresse MAC de 192.168.2.12 pour initier un ping.
- 192.168.2.16 envoie un broadcast ARP pour connaître l’adresse MAC de 192.168.2.12.
- 192.168.2.12 répond avec son adresse MAC.
- 192.168.2.16 envoie un message ICMP Echo (ping) à 192.168.2.12, qui répond immédiatement.
- Ce ping est répété quatre fois.

